#### get all pinyin names for cities with weather information

In [70]:
# -*- coding: utf-8 -*-
import os
def get_all_cities():
    all_cities = set()
    files = os.listdir("./weather/city_weather/")
    for f in files:
        all_cities.add(f)
    return all_cities

In [82]:
print get_all_cities()

set(['dongying', 'taizhou', 'xinyang', 'zhengzhou', 'liaoyang', 'rizhao', 'huanggang', 'puyang', 'wuhu', 'huangshi', 'deyang', 'xiangyang', 'sanming', 'huaian', 'fushun', 'xiamen', 'sanya', 'weihai', 'nanning', 'yangzhou', 'foshan', 'guiyang', 'hangzhou', 'fuzhou', 'shaoxing', 'wuxi', 'jingzhou', 'ankang', 'huizhou', 'yulin', 'tongliao', 'jinhua', 'bengbu', 'zhangjiakou', 'changsha', 'jining', 'tianjin', 'tianmen', 'tongling', 'xingtai', 'huaibei', 'dongguan', 'jiangmen', 'dazhou', 'nantong', 'guangzhou', 'zhangzhou', 'quzhou', 'wuhan', 'shangrao', 'luoyang', 'xuzhou', 'changzhi', 'qingdao', 'chongqing', 'suqian', 'fuyang', 'zigong', 'leshan', 'changzhou', 'ningde', 'lishui', 'luan', 'xianning', 'shanghai', 'weifang', 'chengdu', 'beijing', 'shenyang', 'nanjing', 'qingyuan', 'zhuzhou', 'wuzhou', 'langfang', 'zhenjiang', 'zhongshan', 'shanwei', 'zhuhai', 'kunming', 'jiaxing', 'xiaogan', 'xianyang', 'wenzhou', 'zhoushan', 'shijiazhuang', 'liuzhou', 'ningbo', 'yichang', 'suzhou', 'nanping'

#### find the day range of shop trends

In [27]:
from datetime import datetime 
def find_earliest_day_of_shop_trend():
    folder = "/home/hao/Dropbox/dataset/Analysis/PayTrend/"
    filename_template = "CustomerFlow_%s.csv"
    minday = datetime.strptime("2020-12-12",'%Y-%m-%d').date()
    maxday = datetime.strptime("1770-12-12",'%Y-%m-%d').date()
    for i in xrange(1,2001):
        filename = filename_template%i
        filename = os.path.join(folder, filename)
        with open(filename,'r') as fp:
            lines = fp.readlines()
            firstlline = lines[0]
            lastline = lines[-1]
            day = datetime.strptime(firstlline.split(',')[0],'%Y-%m-%d').date()
            if day<minday:
                minday = day
            day = datetime.strptime(lastline.split(',')[0],'%Y-%m-%d').date()
            if day>maxday:
                maxday=day
    return minday,maxday
minday,maxday = find_earliest_day_of_shop_trend()
print minday,maxday

2015-06-26 2016-10-31


### do some statistics abou the number of days that are not covered by the weather. i.e. find the number of days that are before 2015-07-01

In [26]:
import operator
def find_invalid_days_for_shop(i):
    threhold_day = datetime.strptime("2015-07-01",'%Y-%m-%d').date()
    folder = "/home/hao/Dropbox/dataset/Analysis/PayTrend/"
    filename_template = "CustomerFlow_%s.csv"
    filename = filename_template % i
    filename = os.path.join(folder,filename)
    days_res = []
    with open(filename,'r') as fp:
        lines = fp.readlines()
        for l in lines:
            day = datetime.strptime(l.split(',')[0],'%Y-%m-%d').date()
            if day>=threhold_day:
                break
            else:
                days_res.append(day)
    return days_res

def find_invalid_days_for_each_shop():
    dic_res = {}
    for i in xrange(1,2001):
        number_of_invalid_days = len(find_invalid_days_for_shop(i))
        if number_of_invalid_days > 0:
            dic_res[i] = number_of_invalid_days
    sorted_x = sorted(dic_res.items(), key=operator.itemgetter(0))
    for pair in sorted_x:
        print 'shop_id:',pair[0],"  days_without_weather:",pair[1]
find_invalid_days_for_each_shop()

shop_id: 7   days_without_weather: 3
shop_id: 88   days_without_weather: 2
shop_id: 145   days_without_weather: 5
shop_id: 200   days_without_weather: 2
shop_id: 217   days_without_weather: 3
shop_id: 260   days_without_weather: 5
shop_id: 304   days_without_weather: 2
shop_id: 315   days_without_weather: 4
shop_id: 341   days_without_weather: 1
shop_id: 345   days_without_weather: 2
shop_id: 358   days_without_weather: 2
shop_id: 375   days_without_weather: 4
shop_id: 380   days_without_weather: 3
shop_id: 386   days_without_weather: 3
shop_id: 391   days_without_weather: 2
shop_id: 404   days_without_weather: 2
shop_id: 455   days_without_weather: 1
shop_id: 531   days_without_weather: 2
shop_id: 548   days_without_weather: 4
shop_id: 560   days_without_weather: 5
shop_id: 568   days_without_weather: 1
shop_id: 668   days_without_weather: 2
shop_id: 676   days_without_weather: 2
shop_id: 750   days_without_weather: 4
shop_id: 850   days_without_weather: 3
shop_id: 897   days_without_

### how many different weather conditions and wind levels

In [62]:
import pandas as pd
def all_city_weather():
    files = os.listdir("./weather/city_weather/")
    all  = []
    invalids = []
    for f in files:
        try:
            weather_file = os.path.join("./weather/city_weather",f)
            cur= pd.read_csv(weather_file,names=['day','high_temp','low_temp','weather','wind','wind_level'])
            all.append(cur)
        except:
            invalids.append(f)
    all = pd.concat(all,ignore_index=True)
    return all,invalids
all_weather_csv,invalids = all_city_weather()

In [68]:
weather_conditions = all_weather_csv['weather'].value_counts()
weather_conditions.to_csv('weather_consitions.csv')

In [69]:
wind_level = all_weather_csv['wind_level'].value_counts()
wind_level.to_csv('wind_levels.csv')

### generate vector for each weather condition

In [102]:
def generate_vector_for_each_weater_condition():
    all_weather_conditions = ['晴天', '多云',	'阴天','小雨','中雨','大雨','阵雨','暴雨','雾霾','小雪','中雪','大雪','阵雪','沙尘']
    file_ = './weather/weather_consitions_processed.csv'
    dic = {}
    with open(file_,'r') as fp:
        lines = fp.readlines()
        for l in lines:
            l = l.strip('\r\n,')
            terms = l.split(',')
            key_ = terms[0]
            values = terms[2:]
            vector = [0]*len(all_weather_conditions)
            for v in values:
                try:
                    vector[all_weather_conditions.index(v)] = 1
                except:
                    print v,key_
            dic[key_]=vector
    return dic
dic = generate_vector_for_each_weater_condition()

### get dictionary mapping day to weather vector for each city

In [107]:
def get_map_from_city_to_daily_weather():
    weather_to_vector = generate_vector_for_each_weater_condition()
    res = {}
    cityfiles = os.listdir("./weather/city_weather/")
    for city in cityfiles:
        sub_dict = res.setdefault(city,{})
        with open(os.path.join("./weather/city_weather/",city),'r') as fp:
            lines = fp.readlines()
            for l in lines:
                terms = l.split(',')
                day = datetime.strptime(terms[0],'%Y-%m-%d').date()
                temps = map(int,terms[1:3])
                weather = weather_to_vector[terms[3]]
                vector = temps + weather
                sub_dict[day] = vector
    return res
city_to_daily_weather = get_map_from_city_to_daily_weather()

In [111]:
pickle.dump(city_to_daily_weather,open('city_to_daily_weather.pkl','wb'))

In [112]:
city_to_daily_weather['beijing']

{datetime.date(2015, 7, 1): [33, 18, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 2): [34, 18, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 3): [33, 20, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 4): [27, 19, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 5): [30, 21, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 6): [31, 22, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 7): [33, 22, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 8): [33, 22, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 9): [34, 23, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 datetime.date(2015, 7, 10): [32,
  23,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 datetime.date(2015, 7, 11): [35,
  24,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 datetime.date(2015, 7, 12): [38,
  25,
  1,
  0,
  0,
  0

### get cityname for each shop

In [100]:
from pypinyin import lazy_pinyin
def get_pinyin(chs_name):
    '''
    return the pinyin in str type for  given  Chinese words in type str
    '''
    if chs_name == '泰州':
        return 'taizhou2'
    elif chs_name == '长治':
        return 'changzhi'
    elif chs_name == '厦门':
        return 'xiamen'
    elif chs_name == '六安':
        return 'luan'
    elif chs_name == '蚌埠':
        return 'bengbu'
    else:
        return reduce(lambda x, y: x + y, lazy_pinyin(chs_name.decode('utf-8'))).encode('utf-8')
def get_shop_map_to_cityname():
    dicres= {}
    FILE = '/home/hao/Dropbox/dataset/shop_info.csv'
    with open(FILE, 'r') as fp:
        lines = fp.readlines()
        for l in lines:
            terms = l.split(',')
            shop = int(terms[0])
            city = get_pinyin(terms[1])
            dicres[shop] = city
    return dicres
maps = get_shop_map_to_cityname()

In [113]:
maps[12]

'ningbo'

In [110]:
import pickle
pickle.dump(maps,open('shop_2_city.pkl','wb'))

####check if each city is in the weather city list, no output means yes

In [126]:
def check():
    for i in xrange(1,2001):
        if maps[i] not in all_aqi_cities:
            print i, maps[i]
check()

### Test for AQI

In [150]:
def get_map_from_city_to_daily_aqi():
    aqi_all_data = '/home/hao/Dropbox/dataset/Scripts/weather/aqistudy-master/aqi/alldata.csv'
    all_data = pd.read_csv(aqi_all_data,parse_dates=[0])
    indexes = all_data.columns
    days = all_data['Date']
    res = {}
    #print indexes
    for city in indexes:
        #print city
        pinyin = get_pinyin(city)
        subdict = res.setdefault(pinyin,{})
        aqis = all_data[city]
        pairs = zip(days,aqis)
        for p in pairs:
            subdict[p[0].date()]=p[1]
    res.pop('Date')
    return res
res = get_map_from_city_to_daily_aqi()

In [152]:
pickle.dump(res,open('city_to_daily_aqi.pkl','wb'))

In [158]:
city_to_daily_weather[maps[1]][datetime.strptime("2015-11-13",'%Y-%m-%d').date()]

[16, 11, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [159]:
maps[1]

'huzhou'